In [9]:
import boto3
import json

bedrock_agent = boto3.client("bedrock-agent", region_name="us-east-1")
bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")
MODEL_ID = "amazon.nova-micro-v1:0"

In [15]:
temperature = .7

inference_config = {"temperature": temperature}

system_prompts = [{"text": "You are a virtual travel assistant that suggests festivals around the world based on user preferences."
                + "Only return festival names and a brief description."}]

messages = []

message_1 = {
    "role": "user",
    "content": [{"text": "Create a list of 3 famous festivals."}]
}

messages.append(message_1)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

def print_response(response):
    model_response = response.get('output', {}).get('message', {}).get('content', [{}])[0].get('text', '')

    print("✈️ Your suggested travel destinations:")
    print(model_response)

print_response(response)

✈️ Your suggested travel destinations:
1. **Carnival (Rio de Janeiro, Brazil)**  
   A vibrant and colorful celebration featuring parades, samba music, dancing, and elaborate costumes.

2. **Diwali (India)**  
   The Festival of Lights, celebrated with fireworks, lamps, and sweets, symbolizing the victory of light over darkness.

3. **Oktoberfest (Munich, Germany)**  
   An annual festival featuring beer, traditional Bavarian food, music, and costumed dancers, celebrating the harvest season.


In [16]:
message_2 = {
        "role": "user",
        "content": [{"text": "Only suggest festivals that are no more than one short flight away."}]
}

messages.append(message_2)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
Sure, here are three famous festivals that are within a short flight from many parts of the world:

1. **Carnival in Rio de Janeiro, Brazil**
   - A vibrant and colorful festival featuring parades, samba music, dancing, and elaborate costumes.

2. **Holi Festival of Colors, India**
   - Celebrated with the throwing of colored powders, music, dancing, and feasting, marking the arrival of spring.

3. **Oktoberfest, Munich, Germany**
   - A world-famous festival featuring beer tents, traditional Bavarian food, music, and folk costumes.


In [17]:
message_3 = {
        "role": "user",
        "content": [{"text": "Festivals that are near the mountains."}]
}

messages.append(message_3)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
1. **Festival de la Luz (Mexico City, Mexico)**
   - A dazzling celebration of light, music, and technology held in the historic Zócalo square, featuring stunning light installations and performances by renowned artists.

2. **Holi (Various Locations in India)**
   - Though not always near mountains, Holi is celebrated in various regions including the hill station of Manali, featuring vibrant colors and joyous festivities.

3. **La Tomatina (Buñol, Spain)**
   - Held in a small town near Valencia, this famous festival involves a massive tomato fight, drawing participants from all over the world. Though not mountainous, it’s a short flight from major Spanish cities.


In [13]:
try:
    response = bedrock_agent.create_prompt(
        name="Travel-Agent-Prompt",
        description="Checks if all trip information has been provided.",
        variants=[
            { 
                "name": "Variant1",
                "modelId": MODEL_ID,
                "templateType": "CHAT",
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.4
                    }
                },
                "templateConfiguration": { 
                    "chat": {
                        'system': [ 
                            {
                                "text": """You are a travel agent evaluating trip requests for custom itineraries. 
                                Review the message carefully and answer YES or NO to the following screening questions. 
                                Be strict—if any detail is missing or unclear, answer NO.

                                A) Is the destination clearly stated?
                                B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
                                C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
                                D) Is there any mention of a valid passport or travel documentation?
                                E) Is there enough information to follow up with a proposed itinerary?"""
                            }
                        ],
                        'messages': [{
                            'role': 'user',
                            'content': [ 
                                {
                                    'text': "Trip request: {{event_request}}"
                                }
                            ]
                        }],
                        'inputVariables' : [
                            { 'name' : 'event_request'}
                        ]
                    }
                }
        }]
    )
    print("Created!")
    prompt_arn = response.get("arn")
except bedrock.exceptions.ConflictException as e:
    print("Already exists!")
    response = bedrock.list_prompts()
    prompt = next((prompt for prompt in response['promptSummaries'] if prompt['name'] == "TripBooker_xyz"), None)
    prompt_arn = prompt['arn']

prompt_arn

Created!


'arn:aws:bedrock:us-east-1:982248023567:prompt/HXHMG8WXI2'

In [19]:
response = bedrock.converse(
    modelId=prompt_arn,
    promptVariables={
        'event_request': {
            'text': """
                Hi there! I'm planning a trip to Germany with my partner and would love some help organizing the itinerary. We're hoping to travel between September 10–20 this year, ideally flying into Rome and spending a few days in Berlin and Bonn as well. We’d love recommendations on tours, cultural sites, and good local restaurants. We’re not interested in anything risky like skydiving or hiking remote trails — just want a relaxing and enriching experience. We both have valid passports. Let me know what other details you need!
                """
        }
    },
)
print(response['output']['message']['content'][0]['text'])

YES

Here's the evaluation based on the screening questions:

A) Is the destination clearly stated?
- YES. The destination is clearly stated as Germany.

B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
- YES. The travel dates are within a reasonable range (September 10–20 this year).

C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
- YES. The request specifies a desire for a relaxing and enriching experience, avoiding risky activities like skydiving or remote hiking.

D) Is there any mention of a valid passport or travel documentation?
- YES. Both travelers have valid passports.

E) Is there enough information to follow up with a proposed itinerary?
- YES. The request provides sufficient details including the destination, travel dates, cities to visit (Berlin and Bonn), and preferences for tours, cultural sites, and local restaurants.

Therefore, the request is complete and clear.
